# **Chatbot copilot**

In [1]:
!pip -q install langchain

In [2]:
!pip -q install duckduckgo-search

## Data retriever using DuckDuckgo

### DuckDuckGo with Langchain

In [9]:
from langchain.tools import DuckDuckGoSearchRun

In [10]:
search = DuckDuckGoSearchRun()

In [11]:
search.run("llmops")

'Learn how to operationalize and manage large language models using Azure ML, a MLOps platform that enables the automation of tasks such as data preparation, model discovery, tuning, deployment, and monitoring. Explore the benefits and challenges of using LLMs in real-world applications and the risks of using them responsibly. LLMOps is a process of managing Large Language Models (LLMs) and LLM-powered applications using tools and best practices. It differs from MLOps in terms of tools, methods, and goals. Learn how LLMOps facilitates the development, deployment, and maintenance of LLMs using prompt engineering, fine-tuning, RAG, and more. Learn what LLMOps is, how it works, and why it is important for the deployment and maintenance of large language models (LLMs). Find out the steps, techniques, and challenges of LLMOps, such as selection of foundation models, adaptation to downstream tasks, evaluation, and deployment and monitoring. Compare LLMOps with MLOps and see examples of popul

In [15]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=2)

In [13]:
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")

In [14]:
search.run("llmops")

"[snippet: Automated Testing for LLMOps. Learn how LLM-based testing differs from traditional software testing and implement rules-based testing to assess your LLM application. Build model-graded evaluations to test your LLM application using an evaluation LLM. Automate your evals (rules-based and model-graded) using continuous integration tools from ..., title: Automated Testing for LLMOps 03：Comprehensive Testing Framework ..., link: https://blog.csdn.net/shizheng_Li/article/details/136271559], [snippet: All CI/CD pipelines in CircleCI are defined in a single YAML file at the path .circleci/config.yml in your project's Git repository. This tutorial explains the example configuration file used throughout the course. There are three key components to every CircleCI config file: jobs, commands, and workflows., title: Automated Testing for LLMOps 04：Exploring the CircleCI config file, link: https://blog.csdn.net/shizheng_Li/article/details/136271640], [snippet: 为了帮助每一个程序员掌握以上 AI 大模型开发的12

### DuckDuckGo API

In [17]:
from duckduckgo_search import DDGS

In [16]:
'''
  "title": result["title"],
  "body": result["body"],
  "link": result["href"],
'''

with DDGS() as ddgs:
    results = [r for r in ddgs.text('what is llmops?', safesearch='on', timelimit='y', max_results=3)]

number_of_result = 0
snippets = []

for result in results:
  number_of_result +=1
  title = result['title']
  body = result['body']
  link = result['href']

  snippet = {'title': title, 'body': body, 'link': link}

  snippets.append(snippet)

In [18]:
print(snippets[0]['body'])

Leverage MLOps for Large Language Models, i.e., LLMOps: Over the years, MLOps has demonstrated its ability to enhance the development, deployment, and maintenance of ML models, leading to more agile and efficient machine learning systems.


**Wrap it in a function**

In [19]:
def DDGS_RAG (query, max_results=2):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, safesearch='on', timelimit='y', max_results=max_results)]

  snippets = []
  for result in results:
    title = result['title']
    body = result['body']
    link = result['href']
    snippet = {'title': title, 'body': body, 'link': link}
    snippets.append(snippet)

  return snippets

In [20]:
res = DDGS_RAG('what is python ?')

In [21]:
res

[{'title': "What Is Python Used For? A Beginner's Guide | Coursera",
  'body': "Python is a computer programming language often used to build websites and software, automate tasks, and conduct data analysis. Python is a general-purpose language, meaning it can be used to create a variety of different programs and isn't specialized for any specific problems.",
  'link': 'https://www.coursera.org/articles/what-is-python-used-for-a-beginners-guide-to-using-python'},
 {'title': 'What is Python? - GeeksforGeeks',
  'body': 'Python is a high-level, general-purpose, and interpreted programming language used in various sectors including machine learning, artificial intelligence, data analysis, web development, and many more. Python is known for its ease of use, powerful standard library, and dynamic semantics.',
  'link': 'https://www.geeksforgeeks.org/what-is-python/'}]

## Chatbot using Google Gemini

In [32]:
import google.generativeai as genai
from dotenv import load_dotenv

import os

In [36]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [37]:
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

In [61]:
result = model.generate_content('what is llmops?')

In [62]:
print(result.text)

LLMOPS stands for **Lost Laboratory of Kwalish**. It is a tabletop role-playing game adventure module published by Wizards of the Coast in 2014. It is designed for use with the fifth edition of the Dungeons & Dragons role-playing game.

The adventure is set in the Forgotten Realms campaign setting and is designed for a party of four to six characters of levels 1-3. The players take on the role of adventurers who are tasked with finding a lost laboratory that belongs to the wizard Kwalish. The laboratory is said to contain powerful magical artifacts and secrets that could be used for good or evil.

The adventure is divided into three parts. In the first part, the players must travel to the town of Parnast and gather information about the laboratory. In the second part, the players must travel to the laboratory and explore it. In the third part, the players must defeat the evil forces that are guarding the laboratory and retrieve the magical artifacts.

LLMOPS is a well-written and excit

In [63]:
def Gemini_generation(request, context):
  prompt = f"""Your main task is to respond to the request using the context provided.
  The context is indicated with the tag 'context' and the request is indicated with
  the tag 'request'.
  If the context doesn't have relevant information about the request, or if
  the request is a basic and very simple one, you can then respond using your own
  knowledge.
  Here's some examples of a basic request: 'hello', 'whats 1+1?', 'who are you?'
  If the knowledge you have contradicts the context provided. Then use the context
  information.

  Context:{context}
  Request:{request}
  """
  response = model.generate_content(prompt,
                                    generation_config=genai.types.GenerationConfig(
                                    candidate_count=1,
                                    max_output_tokens=500,
                                    temperature=0.1))
  return response

In [64]:
def Gemini_digest (list_of_context, list_of_titles):
  prompt = f""" Your task is to generate a text summary based on the provided data.
  To complete this task successfully you need to follow these steps:

  1- Generate seperate summaries for each text in the provided list indicated with the tag 'list_of_context'.
  2- Understand the context of each summary and generate seperate summaries.
  3- The final summary must be in bullet points, add a title for each bulet point, titles must be from the provided list indicated with the tag 'list_of_titles'.
  4- Generate a good title for this complete and unified summary.

  make sure the output is clear and easy to read.

  list_of_context: {list_of_context}.
  list_of_titles: {list_of_titles}.
  """
  response = model.generate_content(prompt,
                                    generation_config=genai.types.GenerationConfig(
                                    candidate_count=1,
                                    max_output_tokens=500,
                                    temperature=0.2))
  return response

In [65]:
def Chatbot (query):
  # Call DDGS_RAG function to retrieve the data
  source_data = DDGS_RAG(query)
  answers=[]
  links=[]
  titles=[]
  ref=[]

  # Loop on the retrieved data and generate responses with Gemini_generation function
  for data in source_data:
    response = Gemini_generation(query, data['body'])
    answers.append(response.text)
    links.append(data['link'])
    titles.append(data['title'])

  # Reference creation (titles, link)
  for i in range(len(links)):
    ref.append(titles[i]+' :\n'+links[i])


  # Call the Gemini_digest function to return the final answer
  summary = Gemini_digest (answers, titles)

  digest = summary.text + '\n\n'+'References: \n\n' + "\n".join(ref)
  return digest

In [67]:
res = Chatbot('what is llmops?')
print(res)

Title: Understanding LLMOps: Operationalizing and Managing Large Language Models

* **LLMOps Overview:**
    * LLMOps (Large Language Model Operations) is a specialized branch of MLOps focused on the unique challenges of developing, deploying, and maintaining large language models (LLMs).
    * LLMs are AI models trained on massive text data and can perform various language-related tasks, such as text generation, translation, summarization, and question answering.

* **Key Aspects of LLMOps:**
    * **Data Management:**
        * LLMs require vast amounts of high-quality text data for training.
        * LLMOps involves managing and curating this data, ensuring its accuracy, consistency, and relevance to the LLM's intended tasks.
    * **Model Training:**
        * Training LLMs is computationally intensive and can take weeks or months.
        * LLMOps provides tools and techniques for optimizing the training process, managing resources, and monitoring progress.
    * **Deployment and

## Stramlit Application

In [75]:
print("\n\n".join(['retjk', 'kjtkler']))

retjk

kjtkler


## Future improvements

In [143]:
res = Chatbot('hello')
print(res)

Title: Greetings and Salutations

* **Formal Greetings:**
    * "Greetings!" is a formal way to say hello, often used in professional or academic settings.
    * "Hello!" is a more casual greeting, suitable for most social situations.
    * "Good morning/afternoon/evening" is a polite way to greet someone at a specific time of day.
    * "How do you do?" is a formal way to inquire about someone's well-being.

* **Informal Greetings:**
    * "Hey!" or "Hi!" are common informal greetings among friends and acquaintances.
    * "What's up?" or "Sup?" are casual ways to ask someone how they are doing.
    * "Yo!" is a slang greeting, often used among young people.
    * "Peace out!" or "Later!" are informal ways to say goodbye.

* **Songs About Saying Hello:**
    * "Hello" by Lionel Richie is a classic song about the joy of greeting someone you love.
    * "Hello, Goodbye" by The Beatles is a song about the bittersweet nature of saying hello and goodbye.
    * "Hello, Dolly!" by Louis Arms